<a href="https://colab.research.google.com/github/BharatXBhalla/Deep-Learning-CDAC/blob/main/Transfer_Learning/Transfer_Learning_Monkey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions 10-monkey-species

usage: kaggle competitions [-h]
                           {list,files,download,submit,submissions,leaderboard}
                           ...
kaggle competitions: error: argument command: invalid choice: '10-monkey-species' (choose from 'list', 'files', 'download', 'submit', 'submissions', 'leaderboard')


In [6]:
! kaggle datasets download -d slothkong/10-monkey-species

 99% 539M/547M [00:03<00:00, 185MB/s]
100% 547M/547M [00:03<00:00, 184MB/s]


In [7]:
from tensorflow.keras.applications import MobileNet

img_row,img_cols=224,224

MobileNet = MobileNet(weights="imagenet",include_top=False,input_shape=(img_row,img_cols,3))

for layers in MobileNet.layers:
    layers.trainable=False

17235968/17225924 [==============================] - 0s 0us/step


In [8]:
import tensorflow as tf

def addTopModelMobileNet(bottomModel,num_classes):

  top_model = bottomModel.output
  top_model = tf.keras.layers.GlobalAveragePooling2D()(top_model)
  top_model = tf.keras.layers.Dense(1024,activation="relu")(top_model)
  top_model = tf.keras.layers.Dense(1024,activation="relu")(top_model)
  top_model = tf.keras.layers.Dense(512,activation="relu")(top_model)
  top_model = tf.keras.layers.Dense(num_classes,activation="softmax")(top_model)

  return top_model
  
  
  

In [9]:
FC_Head = addTopModelMobileNet(MobileNet,10)

In [10]:
from keras.models import Model

In [11]:
model = Model(inputs=MobileNet.input,outputs=FC_Head)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [13]:
!unzip 10-monkey-species.zip

Archive:  10-monkey-species.zip
  inflating: monkey_labels.txt       
  inflating: training/training/n0/n0018.jpg  
  inflating: training/training/n0/n0019.jpg  
  inflating: training/training/n0/n0020.jpg  
  inflating: training/training/n0/n0021.jpg  
  inflating: training/training/n0/n0022.jpg  
  inflating: training/training/n0/n0023.jpg  
  inflating: training/training/n0/n0024.jpg  
  inflating: training/training/n0/n0025.jpg  
  inflating: training/training/n0/n0026.jpg  
  inflating: training/training/n0/n0027.jpg  
  inflating: training/training/n0/n0028.jpg  
  inflating: training/training/n0/n0029.jpg  
  inflating: training/training/n0/n0030.jpg  
  inflating: training/training/n0/n0031.jpg  
  inflating: training/training/n0/n0032.jpg  
  inflating: training/training/n0/n0033.jpg  
  inflating: training/training/n0/n0034.jpg  
  inflating: training/training/n0/n0035.jpg  
  inflating: training/training/n0/n0036.jpg  
  inflating: training/training/n0/n0037.jpg  
  inflatin

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=45,width_shift_range=0.3,height_shift_range=0.3,horizontal_flip=True,fill_mode="nearest")

In [16]:
validation_datagen=ImageDataGenerator(rescale=1./255)

In [17]:
batch_size=25

In [21]:
train_generator = train_datagen.flow_from_directory(
    '/content/training/training',
    target_size=(img_row,img_cols),
    batch_size=batch_size,
    class_mode="categorical"
    
)

Found 1098 images belonging to 10 classes.


In [22]:
validation_datagen = validation_datagen.flow_from_directory(
    '/content/validation/validation',
    target_size=(img_row,img_cols),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 272 images belonging to 10 classes.


In [25]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [26]:
checkpoint = ModelCheckpoint("/content/models/monkey.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

In [27]:
earlyStopping = EarlyStopping(monitor="val_loss",
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)

In [28]:
callbacks=[checkpoint,earlyStopping]

In [29]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(lr=0.001),metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [30]:
# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 32

In [31]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_datagen,
    validation_steps = nb_validation_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/5
34/34 [==============================] - ETA: 0s - loss: 2.5064 - accuracy: 0.3424
Epoch 1: val_loss improved from inf to 0.90949, saving model to /content/models/monkey.h5
34/34 [==============================] - 71s 2s/step - loss: 2.5064 - accuracy: 0.3424 - val_loss: 0.9095 - val_accuracy: 0.7050
Epoch 2/5
34/34 [==============================] - ETA: 0s - loss: 1.0372 - accuracy: 0.7028
Epoch 2: val_loss improved from 0.90949 to 0.30036, saving model to /content/models/monkey.h5
34/34 [==============================] - 67s 2s/step - loss: 1.0372 - accuracy: 0.7028 - val_loss: 0.3004 - val_accuracy: 0.8850
Epoch 3/5
34/34 [==============================] - ETA: 0s - loss: 0.6362 - accuracy: 0.7854
Epoch 3: val_loss did not improve from 0.30036
34/34 [==============================] - 67s 2s/step - loss: 0.6362 - accuracy: 0.7854 - val_loss: 0.3222 - val_accuracy: 0.8750
Epoch 4/5
34/34 [==============================] - ETA: 0s - loss: 0.4626 - accuracy: 0.8647
Epoch 4: v

In [33]:
from keras.models import load_model

classifier = load_model('/content/models/monkey.h5')